In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
import cv2
import numpy as np
import random as rn
import pandas as pd
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
df=pd.read_csv(r"E:\machinelearning\data\train.csv")
a,b=np.unique(df["target"],return_counts=True)
from sklearn.model_selection import train_test_split

train,valid=train_test_split(df,test_size=0.2)
train,test=train_test_split(train,test_size=0.01)

train["image_name_jpg"]=train["image_name"]+".jpg"
test["image_name_jpg"]=test["image_name"]+".jpg"
valid["image_name_jpg"]=valid["image_name"]+".jpg"

train["target"]=train["target"].astype(str)
test["target"]=test["target"].astype(str)
valid["target"]=valid["target"].astype(str)

In [11]:
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    horizontal_flip=True
    )
test_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255) # we have to divide by 255 for testing in android app


train_generator =train_datagen.flow_from_dataframe(
    dataframe=train,
    directory=r"E:/machinelearning/data/train/",
    x_col="image_name_jpg", # name +".jpg"
    y_col="target",
    target_size=(64,64),
    batch_size=32,
    class_mode="binary"
)

Found 26235 validated image filenames belonging to 2 classes.


In [12]:
validation_generator =test_datagen.flow_from_dataframe(
    dataframe=valid,
    directory=r"E:/machinelearning/data/train/",
    x_col="image_name_jpg",
    y_col="target",
    target_size=(64,64),
    batch_size=16,
    class_mode="binary"
)

Found 6626 validated image filenames belonging to 2 classes.


In [13]:
# Part 2 - Building the CNN
# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32,padding="same",kernel_size=3, activation='relu', strides=2, input_shape=[64, 64, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
#cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
## For Binary Classification
cnn.add(Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.01),activation
             ='linear'))

In [14]:
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               262272    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [15]:
# Part 3 - Training the CNN

# Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'hinge', metrics = ['accuracy'])

# Training the CNN on the Training set and evaluating it on the Test set
r=cnn.fit(x = train_generator, validation_data = validation_generator, epochs = 3, verbose = 2)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 820 steps, validate for 415 steps
Epoch 1/3
820/820 - 55s - loss: 0.0778 - accuracy: 0.8819 - val_loss: 0.0492 - val_accuracy: 0.8814
Epoch 2/3
820/820 - 39s - loss: 0.0440 - accuracy: 0.8827 - val_loss: 0.0438 - val_accuracy: 0.8814
Epoch 3/3
820/820 - 39s - loss: 0.0396 - accuracy: 0.8827 - val_loss: 0.0386 - val_accuracy: 0.8814
